In [2]:
# Load Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import gc
import datetime as dt
from datetime import datetime, timedelta 
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import seaborn as sns
import collections as co
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from ipywidgets import widgets, interactive
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
from itertools import cycle
from fastprogress import master_bar, progress_bar

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
current_path = os.getcwd()
use_datetime = 'BaseCase'

test_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Test\\' ## Refrac만 Testdataset으로 할 때 해당 경로를 바꿔주면 됨

def Set_Var(Choose_num_of_first_output_parameter, Choose_num_of_second_output_parameter, use_or_not_ProdBOE, use_or_not_DailyRate, use_or_not_CumProdBOE, 
use_or_not_OilProd, use_or_not_OilRate, use_or_not_CumOil, use_or_not_GasProd, use_or_not_GasRate, use_or_not_CumGas, 
use_or_not_WaterProd, use_or_not_CumMonth, use_or_not_ProdDays, use_or_not_CumProdDay, use_or_not_Shutin, use_or_not_Refrac):

    num_of_features = use_or_not_ProdBOE + use_or_not_DailyRate + use_or_not_CumProdBOE + use_or_not_OilProd + use_or_not_OilRate + use_or_not_CumOil + use_or_not_GasProd + use_or_not_GasRate + use_or_not_CumGas + use_or_not_WaterProd + use_or_not_CumMonth + use_or_not_ProdDays + use_or_not_CumProdDay + use_or_not_Shutin + use_or_not_Refrac
    tmp_1 = [use_or_not_ProdBOE, use_or_not_DailyRate, use_or_not_CumProdBOE, use_or_not_OilProd, use_or_not_OilRate, use_or_not_CumOil, use_or_not_GasProd, use_or_not_GasRate, use_or_not_CumGas, use_or_not_WaterProd, use_or_not_CumMonth, use_or_not_ProdDays, use_or_not_CumProdDay, use_or_not_Shutin, use_or_not_Refrac]
    tmp_2 = ["Prod_BOE", "ProdRate_BOE", "CumProd_BOE", 'LiquidsProd_BBL', 'Liq_rate', 'CumLiquids_BBL', 'GasProd_MCF', 'Gas_rate','CumGas_MCF', 'WaterProd_BBL', "TotalProdMonths", "ProducingDays", 'CumProdDay', "ShutinMonths", "Refrac"]
    used_features = []

    used_features = np.append(used_features, tmp_2[Choose_num_of_first_output_parameter - 1])
    used_features = np.append(used_features, tmp_2[Choose_num_of_second_output_parameter - 1])

    tmp_1 = np.delete(tmp_1, Choose_num_of_first_output_parameter - 1)
    tmp_2 = np.delete(tmp_2, Choose_num_of_first_output_parameter - 1)

    tmp_1 = np.delete(tmp_1, Choose_num_of_second_output_parameter - 2)
    tmp_2 = np.delete(tmp_2, Choose_num_of_second_output_parameter - 2)

    for idx, val in enumerate(tmp_1):
        if val == 1:
            used_features = np.append(used_features, tmp_2[idx])

    name_of_used_features = ''
    for name in used_features:
        name_of_used_features = name_of_used_features + '_' + name
    
    print("Input features =", used_features) 
    print("Onput features =", used_features[0], used_features[1])

    return used_features, name_of_used_features, num_of_features
    

    if not os.path.exists(current_path + '\\Model'):
        os.makedirs(current_path + '\\Model')

###  This function creates a sliding window or sequences of 28 days and one day label ####
###  For Multiple features                                                            ####
# def sliding_windows_mutli_features(data, seq_length):
#     x = []
#     y = []
#     for i in range((data.shape[0])-seq_length-1):
#         _x = data[i:(i+seq_length), :] ## 3 columns for features  
#         _y = data[i+seq_length, 0:2] ## column 0 contains the labbel
#         x.append(_x)
#         y.append(_y)

#     return np.array(x), np.array(y).reshape(-1, 2)

###  This function creates a sliding window or sequences of 28 days and one day label ####
###  For Multiple features                                                            ####
def sliding_windows_mutli_features(data, seq_length):
    x = []
    y = []
    for i in range((data.shape[0])-seq_length):
        _x_prod = data[i:(i+seq_length), 0:2] ## 3 columns for features
        _x_oper = data[i+1:(i+seq_length+1), 2:]
        _x = np.concatenate((_x_prod, _x_oper), axis=1)  
        _y = data[i+seq_length, 0:2] ## column 0 contains the labbel
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y).reshape(-1, 2)

class LSTM2(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM2, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.batch_size = 1
        #self.seq_length = seq_length
        
        self.LSTM2 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout = 0.2)
    
        self.fc1 = nn.Linear(hidden_size, 256)
        self.bn1 = nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.dp1 = nn.Dropout(0.25)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.dp2 = nn.Dropout(0.2)

        self.fc3= nn.Linear(128, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        h_1 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        c_1 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        _, (hn, cn) = self.LSTM2(x, (h_1, c_1))
    
        #print("hidden state shpe is:",hn.size())
        y = hn.view(-1, self.hidden_size)
        
        final_state = hn.view(self.num_layers, x.size(0), self.hidden_size)[-1]
        #print("final state shape is:",final_state.shape)
        
        x0 = self.fc1(final_state)
        x0 = self.bn1(x0)
        x0 = self.dp1(x0)
        x0 = self.relu(x0)
        
        x0 = self.fc2(x0)
        x0 = self.bn2(x0)
        x0 = self.dp2(x0)
        x0 = self.relu(x0)
        
        out = self.fc3(x0)
        #print(out.size())
        return out

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x_data = x
        self.y_data = y

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = Variable(torch.Tensor(self.x_data[idx]))
        y = Variable(torch.Tensor(self.y_data[idx]))
        return x, y

In [3]:
if not os.path.exists(current_path + '\\Results'):
        os.makedirs(current_path + '\\Results')
if not os.path.exists(current_path + '\\Results\\' + use_datetime):
        os.makedirs(current_path + '\\Results\\' + use_datetime)
if not os.path.exists(current_path + '\\Figs\\' + use_datetime):
    os.makedirs(current_path + '\\Figs\\' + use_datetime)
if not os.path.exists(current_path + '\\Figs\\' + use_datetime + '\\TestData'):
    os.makedirs(current_path + '\\Figs\\' + use_datetime + '\\TestData')
if not os.path.exists(current_path + '\\Figs\\' + use_datetime + '\\TestData\\Prod'):
    os.makedirs(current_path + '\\Figs\\' + use_datetime + '\\TestData\\Prod')
if not os.path.exists(current_path + '\\Figs\\' + use_datetime + '\\TestData\\CumProd'):
    os.makedirs(current_path + '\\Figs\\' + use_datetime + '\\TestData\\CumProd')
if not os.path.exists(current_path + '\\Figs\\' + use_datetime + '\\TestData\\ProdbyCum'):
    os.makedirs(current_path + '\\Figs\\' + use_datetime + '\\TestData\\ProdbyCum')

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# current_path = os.getcwd()

if not os.path.exists(current_path + '\\Model'):
        os.makedirs(current_path + '\\Model')

In [4]:
########################### Start of Control Pannel ###########################
"""
    - It should be same with the number of num_features and sum of use_or_not_features.
    - Ex. use_or_not_CumMonth = 1 means you will use that parameter.
"""
use_or_not_ProdBOE    = 0    # Ex. 3000, 1000, 500, ...
use_or_not_DailyRate  = 0    # Ex. 100, 90, 40, ...  = ProdBOE/ProdDays
use_or_not_CumProdBOE = 0    # Ex. 3000, 4000, 4500, ...
use_or_not_OilProd    = 0
use_or_not_OilRate    = 0
use_or_not_CumOil     = 0
use_or_not_GasProd    = 1
use_or_not_GasRate    = 0
use_or_not_CumGas     = 1
use_or_not_WaterProd  = 0 

use_or_not_CumMonth   = 0    # Ex. 1, 2, 3, 4, ...
use_or_not_ProdDays   = 1    # Ex. 30, 31, 29, 5, ...
use_or_not_CumProdDay = 1      
use_or_not_Shutin     = 1    # Ex. 0, 1, 0, 0, 2, 0, ...
use_or_not_Refrac     = 1

Choose_num_of_output_parameter = 2
Choose_num_of_first_output_parameter = 7
Choose_num_of_second_output_parameter = 9

########################### End of Control Pannel ###########################

used_features, name_of_used_features, num_of_features = Set_Var(Choose_num_of_first_output_parameter, Choose_num_of_second_output_parameter, use_or_not_ProdBOE, use_or_not_DailyRate, use_or_not_CumProdBOE, 
                                                                            use_or_not_OilProd, use_or_not_OilRate, use_or_not_CumOil, use_or_not_GasProd, use_or_not_GasRate, use_or_not_CumGas, 
                                                                            use_or_not_WaterProd, use_or_not_CumMonth, use_or_not_ProdDays, use_or_not_CumProdDay, use_or_not_Shutin, use_or_not_Refrac)

Input features = ['GasProd_MCF' 'CumGas_MCF' 'ProducingDays' 'CumProdDay' 'ShutinMonths'
 'Refrac']
Onput features = GasProd_MCF CumGas_MCF


In [5]:
########################### Start of Control Pannel #########################
test_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Test\\' ## Refrac만 Testdataset으로 할 때 해당 경로를 바꿔주면 됨
use_datetime = 'BaseCase' # Check the data in the Data dictionary.
seq_length = 3
seq_length_string = '3'
used_seq = 3
use_or_not_EUR_until_Last_months = 1 ## if 1 --> EUR_standard set by last months automatically. and then, below value(EUR standard) is no used.
used_period_lst = [3, 6, 9] # used_period는 seq_length보다 짧아선 안됨. min of used_period = seq_length
used_period = used_period_lst[0] # Best, Worst Case 그릴때 얼마까지 사용해서 그릴것인지 결정
EUR_standard = 60
choose_scaler = 2  # 1=standard, 2=minmax, 3=Robust, 4=QuantileTransformer, 5=PowerTransformer
lamda = 1 # 여기서는 prediction 할 때 EUR을 계산하는 비율을 결정하는걸로, 학습시켰을 때 비랑 일정하게 하려고 함.
############################## Hyperparameters ##############################
learning_rate = 1e-3
hidden_size = 128
num_layers = 3
num_classes = 1
########################### End of Control Pannel ###########################

In [42]:
""" 
Code for checking Result
"""

input_size = num_of_features

lstm = LSTM2(num_classes, input_size, hidden_size, num_layers)
lstm.to(device)

lstm.apply(init_weights)

criterion = torch.nn.MSELoss().to(device)    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor =0.5, min_lr=1e-7, eps=1e-08)

scaler_prod = MinMaxScaler(feature_range=(0, 1))
scaler_cumprod = MinMaxScaler(feature_range=(0, 0.0375))
scaler_oper = MinMaxScaler(feature_range=(0, 1))

test_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Test\\' ## Refrac만 Testdataset으로 할 때 해당 경로를 바꿔주면 됨
test_file_list = os.listdir(test_input_path)

for idx_, val in enumerate(used_period_lst):
    """
    - Control Pannel
    - Set the parameters
    """
    used_period = val   # used_period는 seq_length보다 짧아선 안됨. min of used_period = seq_length
    pred_period = EUR_standard - used_period

    #########################################

    EUR_true_lst = []
    EUR_pred_lst = []
    EUR_true_lst_cum = []
    EUR_pred_lst_cum = []
    EUR_pred_lst_comb = []
    EUR_true_lst_prodbycum = []
    EUR_pred_lst_prodbycum = []
    Wellname_lst = []

    np.random.shuffle(test_file_list)
    for idx in range(len(test_file_list)):
        X = []
        Y = []
        EUR_True = 0
        EUR_Pred = 0
        EUR_sum = 0

        name_of_data = test_file_list[idx]
        df_temp = pd.read_csv(test_input_path + name_of_data)
        df_TEST = pd.DataFrame(columns=used_features)

        if use_or_not_EUR_until_Last_months == 1:
            EUR_standard = len(df_temp)
            pred_period = EUR_standard - used_period
        else: pass
        
        ## 처음부터 데이터 가져올 때, 생산량 외의 정보는 t+1 시점으로 맞춰줘야함
        for x in used_features:
            df_TEST[x] = df_temp[x]
        data = np.array(df_TEST)
        origin_data = data[:used_period, 0]
        origin_data_cum = data[:used_period, 1]
        scaler_prod.fit(data[:used_seq, 0].reshape(-1, 1))
        scaler_cumprod.fit(data[:used_seq, 1].reshape(-1, 1))
        scaler_oper.fit(data[:, 2:])

        scaled_prod = scaler_prod.transform(data[:, 0].reshape(-1, 1))
        scaled_cumprod = scaler_cumprod.transform(data[:, 1].reshape(-1, 1))
        scaled_oper = scaler_oper.transform(data[:, 2:])

        train_data_scaled = np.concatenate((scaled_prod, scaled_cumprod), axis=1)
        train_data_scaled = np.concatenate((train_data_scaled, scaled_oper), axis=1)

        test_data_normalized = train_data_scaled

        if seq_length <= used_period:
            X_test_prod = test_data_normalized[used_period - seq_length:used_period, 0:2]
            X_test_oper = test_data_normalized[used_period - seq_length+1:used_period+1, 2:]
            X_test = np.concatenate((X_test_prod, X_test_oper), axis=1) 
            Y_test = test_data_normalized[used_period:used_period + pred_period, :]
        else:
            print(" used_period is shorter than the length of seq_length, it should be longer.")

        X_test = np.reshape(X_test, (-1, seq_length, num_of_features))
        Y_test = np.reshape(Y_test, (-1, pred_period, num_of_features))

        Xtest = Variable(torch.Tensor(np.array(X_test)))
        Ytest = Variable(torch.Tensor(np.array(Y_test)))
        Ytest_plot = Ytest.data.numpy()

        predict = []
        Xtest_tmp = Xtest

        for pred in range(pred_period):
            lstm.load_state_dict(torch.load(current_path + '\\Model\\Best_model_Case_' + name_of_used_features + '_' + str(seq_length) + 'seq_' + use_datetime + '_Test' + '.pt'))
            lstm.eval()
            test_predict = lstm(Xtest_tmp.to(device)) ## seq_len 넣어서 다음거 하나 나옴, 다시 얘를 포함해서 생산량(Output이랑 동일) 6개 맞추고, 나머지 차원애들은 참값을 불러와줘야함.
            data_predict = test_predict.cpu().data.numpy()
            print(data_predict[0][0])
            prod_for_compare = Y_test.cpu().data.numpy()[0][2, 0]
            prod_for_compare_ = scaler_prod.inverse_transform(prod_for_compare.reshape(-1, 1))
            print(prod_for_compare_)
            cascade_lst = []
            cascade_lst = np.append(cascade_lst, data_predict[0][0])
            cascade_lst = np.append(cascade_lst, data_predict[0][1])
            
            if pred < pred_period-1:
                for idx in range(num_of_features-2):
                    cascade_lst = np.append(cascade_lst, Ytest[:, pred+1, idx+2])

                predict = np.append(predict, np.array(cascade_lst))
                Xtest_tmp = np.append(Xtest_tmp, cascade_lst)
                Xtest_tmp = np.reshape(Xtest_tmp, (-1, seq_length+1, num_of_features))
                Xtest_tmp = Xtest_tmp[:, 1:, :]
                Xtest_tmp = Variable(torch.Tensor(np.array(Xtest_tmp)))

        for idx in range(num_of_features-2):
            cascade_lst = np.append(cascade_lst, Ytest[:, pred, idx+2])

        predict = np.append(predict, np.array(cascade_lst))
        predict = np.reshape(predict, (-1, pred_period, num_of_features))

        # Inverse Normalizaiton
        predict = scaler.inverse_transform(predict.reshape(-1, num_of_features))
        true = scaler.inverse_transform(Ytest_plot.reshape(-1, num_of_features))

        # Cal EUR by Prod
        data_plot_true = np.append(origin_data, true[:, 0])
        data_plot_pred = np.append(origin_data, predict[:, 0])
        name = name_of_data.strip(".csv")

        EUR_sum = EUR_sum + np.abs(np.sum(predict[:, 0]) - np.sum(true[:, 0]))
        EUR_true = np.sum(true[:, 0]) + np.sum(origin_data)
        EUR_pred = np.sum(predict[:, 0]) + np.sum(origin_data)

        EUR_true_lst = np.append(EUR_true_lst, EUR_true)
        EUR_pred_lst = np.append(EUR_pred_lst, EUR_pred)
        Wellname_lst = np.append(Wellname_lst, name)

        # Cal EUR by CumProd
        data_plot_true_cum = np.append(origin_data_cum, true[:, 1])
        data_plot_pred_cum = np.append(origin_data_cum, predict[:, 1])

        EUR_true_cum = data_plot_true_cum[-1]
        EUR_pred_cum = data_plot_pred_cum[-1]

        EUR_true_lst_cum = np.append(EUR_true_lst_cum, EUR_true_cum)
        EUR_pred_lst_cum = np.append(EUR_pred_lst_cum, EUR_pred_cum)

        # Cal EUR by combination with Prod and CumProd by weight for Learning
        EUR_pred_lst_comb = lamda * EUR_pred_lst + (1-lamda) * EUR_pred_lst_cum

        # Cal EUR by Prod by Cum
        data_plot_true_prodbycum = np.zeros(len(data_plot_true_cum))
        data_plot_pred_prodbycum = np.zeros(len(data_plot_pred_cum))
        origin_data_prodbycum = np.zeros(len(origin_data_cum))

        for idx, value in enumerate(data_plot_true_cum):
            if idx == 0:
                data_plot_true_prodbycum[idx] = value
            else:
                data_plot_true_prodbycum[idx] = (value - data_plot_true_cum[idx-1])

        for idx, value in enumerate(data_plot_pred_cum):
            if idx == 0:
                data_plot_pred_prodbycum[idx] = value
            else:
                data_plot_pred_prodbycum[idx] = (value - data_plot_pred_cum[idx-1])

        for idx, value in enumerate(origin_data_cum):
            if idx == 0:
                origin_data_prodbycum[idx] = value
            else:
                origin_data_prodbycum[idx] = (value - origin_data_cum[idx-1])

        # EUR_sum = EUR_sum + np.abs(np.sum(predict[:, 0]) - np.sum(true[:, 0]))
        EUR_true_prodbycum = np.sum(data_plot_true_prodbycum) + np.sum(origin_data_prodbycum)
        EUR_pred_prodbycum = np.sum(data_plot_pred_prodbycum) + np.sum(origin_data_prodbycum)

        EUR_true_lst_prodbycum = np.append(EUR_true_lst_prodbycum, EUR_true_prodbycum)
        EUR_pred_lst_prodbycum = np.append(EUR_pred_lst_prodbycum, EUR_pred_prodbycum)

        if used_period == 3:
            # Plot for Prod
            figure(num=None, figsize=(19, 6), dpi=80, facecolor='w', edgecolor='k')
            plt.plot(data_plot_true, 'b')
            plt.plot(data_plot_pred, 'r')
            plt.plot(origin_data, 'k')
            plt.legend(['True', 'Prediction'], fontsize=21)
            plt.suptitle(name, fontsize=23)
            plt.xticks(fontsize=21)
            plt.yticks(fontsize=21)
            plt.ylabel(ylabel = 'Production', fontsize=21)
            plt.xlabel(xlabel = 'Month', fontsize=21)
            plt.savefig(current_path + '\\Figs\\' + use_datetime + '\\TestData\\Prod\\' + name + '.png')
            plt.close()

            # Plot for CumProd
            figure(num=None, figsize=(19, 6), dpi=80, facecolor='w', edgecolor='k')
            plt.plot(data_plot_true_cum, 'b')
            plt.plot(data_plot_pred_cum, 'r')
            plt.plot(origin_data_cum, 'k')
            plt.legend(['True', 'Prediction'], fontsize=21)
            plt.suptitle(name, fontsize=23)
            plt.xticks(fontsize=21)
            plt.yticks(fontsize=21)
            plt.ylabel(ylabel = 'Production', fontsize=21)
            plt.xlabel(xlabel = 'Month', fontsize=21)
            plt.savefig(current_path + '\\Figs\\' + use_datetime + '\\TestData\\CumProd\\' + name + '.png')
            plt.close()

            # Plot for ProdbyCum            
            figure(num=None, figsize=(19, 6), dpi=80, facecolor='w', edgecolor='k')
            plt.plot(data_plot_true_prodbycum, 'b')
            plt.plot(data_plot_pred_prodbycum, 'r')
            plt.plot(origin_data_prodbycum, 'k')
            plt.legend(['True', 'Prediction'], fontsize=21)
            plt.suptitle(name, fontsize=23)
            plt.xticks(fontsize=21)
            plt.yticks(fontsize=21)
            plt.ylabel(ylabel = 'Production', fontsize=21)
            plt.xlabel(xlabel = 'Month', fontsize=21)
            plt.savefig(current_path + '\\Figs\\' + use_datetime + '\\TestData\\ProdbyCum\\' + name + '.png')
            plt.close()

    colums_EUR = ['name', 'True', 'Pred']
    df_EUR = pd.DataFrame(columns=colums_EUR)
    df_EUR['name'] = Wellname_lst
    df_EUR['True'] = EUR_true_lst
    df_EUR['Pred'] = EUR_pred_lst
    df_EUR.to_csv(current_path + '/Results/' + use_datetime + '/EUR_scatterplot_' + str(val) + '.csv')

    df_EUR_cum = pd.DataFrame(columns=colums_EUR)
    df_EUR_cum['name'] = Wellname_lst
    df_EUR_cum['True'] = EUR_true_lst_cum
    df_EUR_cum['Pred'] = EUR_pred_lst_cum
    df_EUR_cum.to_csv(current_path + '/Results/' + use_datetime + '/EUR_scatterplot_cum_' + str(val) + '.csv')

    df_EUR_comb = pd.DataFrame(columns=colums_EUR)
    df_EUR_comb['name'] = Wellname_lst
    df_EUR_comb['True'] = EUR_true_lst_cum
    df_EUR_comb['Pred'] = EUR_pred_lst_comb
    df_EUR_comb.to_csv(current_path + '/Results/' + use_datetime + '/EUR_scatterplot_comb_' + str(val) + '.csv')

    df_EUR_prodbycum = pd.DataFrame(columns=colums_EUR)
    df_EUR_prodbycum['name'] = Wellname_lst
    df_EUR_prodbycum['True'] = EUR_true_lst_prodbycum
    df_EUR_prodbycum['Pred'] = EUR_pred_lst_prodbycum
    df_EUR_prodbycum.to_csv(current_path + '/Results/' + use_datetime + '/EUR_scatterplot_prodbycum_' + str(val) + '.csv')

3.7121668


AttributeError: 'numpy.ndarray' object has no attribute 'cpu'